In [29]:
import pandas as pd
import numpy as np
import pyreadstat

In [30]:
weight = pd.read_stata('/Users/novak/ZhangYS Dropbox/Shared/HCAP/data/source/mexcog/2016/Master_Follow_up_File_Mex_Cog_2016_WEIGHT.dta') # information about individuals selected for survey
informant = pd.read_stata('/Users/novak/ZhangYS Dropbox/Shared/HCAP/data/source/mexcog/2016/Informant_Interview_Mex_Cog_2016.dta') 
respondent = pd.read_stata('/Users/novak/ZhangYS Dropbox/Shared/HCAP/data/source/mexcog/2016/Cognitive_Assessment_Mex_Cog_2016.dta')
mexhar = pd.read_stata('/Users/novak/ZhangYS Dropbox/Shared/HCAP/data/source/mexcog/2016/Multi-Country Harmonized Factor Scores Mex-Cog 2016.dta')

In [31]:
#data prep and merge

# keep only EAP scores 
mhasfgcp = mexhar[['cunicah', 'np', 'fmem', 'fexf', 'flang', 'forient', 'fgcp']]

# merge with informant data
harm_inf = pd.merge(mhasfgcp, informant, on=['cunicah', 'np'], how='inner')

# merge with respondent data
harm_inf_res = pd.merge(harm_inf, respondent, on=['cunicah', 'np'], how='inner')
mex_cog_df = pd.merge(harm_inf_res, weight, on=['cunicah', 'np'], how='inner')

#create id_mexcog column 
mex_cog_df['id_mexcog'] = mex_cog_df['cunicah'].astype(str) + mex_cog_df['np'].astype(str)





In [32]:
# 1.0 Clean Data 

# Generate new variables 'hrs' and 'mex'
mex_cog_df['hrs'] = 0
mex_cog_df['mex'] = 1

# Create a new column 'wgt' that is a copy of 'factor_mxcog_16'
mex_cog_df['wgt'] = mex_cog_df['factor_mxcog_16']

# Mapping the values for each column
mapping = {
    '1:  Correct Correcto': 1,
    '0:  Incorrect Incorrecto': 0
}

# Orientation to time (4p): year, day, date, month;
mex_cog_df['D1_t3'] = mex_cog_df['D1_t3'].map(mapping).astype(float)
mex_cog_df['D1_t2'] = mex_cog_df['D1_t2'].map(mapping).astype(float)
mex_cog_df['D1_t1'] = mex_cog_df['D1_t1'].map(mapping).astype(float)
mex_cog_df['D1_t4'] = mex_cog_df['D1_t4'].map(mapping).astype(float)

mex_cog_df['r1time'] = mex_cog_df[['D1_t3', 'D1_t2', 'D1_t1', 'D1_t4']].sum(axis=1)

# Orientation to state (1p);
mex_cog_df['r1state'] = mex_cog_df['D1_t9'].map(mapping).astype(float)

# Word recall: immedicate recall (3p);
mex_cog_df['r1imrc3'] = mex_cog_df['D2_t1_correct'].astype(float)

# Word recall: delayed recall (3p);
mex_cog_df['r1dlrc3'] = mex_cog_df['D3_t1_correct'].astype(float)

# Serial Sevens;
mex_cog_df['r1ss7'] = mex_cog_df['D7_t2_correct'].astype(float)

# Object names (2p);

mex_cog_df['D5_t3'] = mex_cog_df['D5_t3'].map(mapping).astype(float)
mex_cog_df['D5_t4'] = mex_cog_df['D5_t4'].map(mapping).astype(float)
mex_cog_df['r1obj'] = mex_cog_df[['D5_t3', 'D5_t4']].sum(axis=1)

# Repeat a sentence (1p);
mex_cog_df['r1rptsen'] = mex_cog_df['D5_t9'].map(mapping).astype(float)

# R follow written command (1p);
mex_cog_df['r1combfol'] = mex_cog_df['D5_t10'].map(mapping).astype(float)

# Executive function (3p);
mex_cog_df['r1exf'] = mex_cog_df['D5_t1_correct'].astype(float)

# Write a sentence (1p);
mex_cog_df['r1senten'] = mex_cog_df['D5_t11'].map(mapping).astype(float)

# Draw picture (1p);
mex_cog_df['r1draw'] = mex_cog_df['D6_t1'].map(mapping).astype(float)

# MMSE (25p);
mex_cog_df['mmse_25'] = mex_cog_df[['r1time', 'r1state', 'r1imrc3', 'r1dlrc3', 'r1ss7', 'r1obj', 'r1rptsen', 'r1combfol', 'r1exf', 'r1senten', 'r1draw']].sum(axis=1)


In [33]:
# 1.1 Respondent Variables;

# age and centered age (within study);
mex_cog_df['age_15'] = mex_cog_df['age_15'].replace(999, pd.NA)
mex_cog_df['rage'] = mex_cog_df['age_15'] #respondent age
mean_rage = mex_cog_df['rage'].mean()
mex_cog_df['rcage'] = mex_cog_df['rage'] - mean_rage #respondent age centered 

# gender
mex_cog_df['rfemale'] = mex_cog_df['sex_15'].map({"1.Male":0, "2.Female":1}).astype("category")

# education
def recode_education(years):
    if years == 0:
        return 0
    elif 1 <= years <= 6:
        return 1
    else:
        return 2

mex_cog_df['red3cat'] = mex_cog_df['yrschool'].apply(recode_education)
mex_cog_df['red3cat1'] = (mex_cog_df['red3cat'] == 0).astype(int) # no education
mex_cog_df['red3cat2'] = (mex_cog_df['red3cat'] == 1).astype(int) # 1-6 years of education
mex_cog_df['red3cat3'] = (mex_cog_df['red3cat'] == 2).astype(int) # 7+ years of education
mex_cog_df['reduc'] = mex_cog_df['yrschool'].apply(lambda x: 17 if x >= 17 else x) # top code education at 17 years

# urban/rural
mex_cog_df['urban'] = mex_cog_df['tam_loc_15'].apply(lambda x: 1 if x == '1.Population = 100,000+' else 0) 


In [34]:
# 1.2 Informant Variables;

# age 
mex_cog_df['iage'] = mex_cog_df['mi_age_16'] #informant age
mex_cog_df['icage'] = mex_cog_df['iage'] - mex_cog_df['iage'].mean() #informant age centered

# female
mex_cog_df['ifemale'] = mex_cog_df['mi_sex_16'].map({"1.Male":0, "2.Female":1}).astype("category")

# education 
mex_cog_df['ieduc'] = mex_cog_df['mi_q58_1c_16'].replace({8: pd.NA, 9: pd.NA}).map(
    {"0. None  Ninguno":0, 
     "1. Elementary  Primaria":6,
     "2. Secondary  Secundaria":9,
     "3. Techincal or commercial  Carrera tecnica o comercial":12,
     "4. Preparatory o high school  Preparatoria o bachillerato":12,
     "5. Basic teaching school  Normal":12, 
     "6. College  Profesional":16, 
     "7. Graduate school  Posgrado":17 
     }) #top code education at 17 years


In [35]:
# 1.3 Relationship;

# relationship
mex_cog_df['relation'] = mex_cog_df['mi_q56_16'].map(
    {"1. Spouse  Conyuge":1,
     "2. Son/daughter  Hijo(a)":2,
     "3. Son/daughter in law  Yerno (nuera)":2,
     "4. Grandchild  Nieto(a)":3,
     "5. Other relative  Otro pariente":4,
     "6. Other    Otro":4
    }
)

mex_cog_df['ispouse'] = (mex_cog_df['relation'] == 1).astype(int) #spouse
mex_cog_df['ichild'] = (mex_cog_df['relation'] == 2).astype(int) #child
mex_cog_df['iothfam'] = (mex_cog_df['relation'] == 3).astype(int) #other family members
mex_cog_df['inonfam'] = (mex_cog_df['relation'] == 4).astype(int) #non-family members


# co-residence
mex_cog_df['coresi'] = mex_cog_df['mi_q57_16'].map({"1. Yes  Si":1, "2. No  No":0}).astype("category")

In [36]:
# ** 2.0 CSI-D;
# ** (1)mi_q2_16 = h1icsicogact1: general decline in mental functioning ;
# ** (2)mi_q3_16 = h1icsicogact2: difficulty remembering things  ;
# ** (3)mi_q4_16 = h1icsicogact3: forgets where put things ;
# ** (4)mi_q5_16 = h1icsicogact4: forgets where things kept;
# ** (5)mi_q6_16 = h1icsicogact5: forgets friends names (1/2);
# ** (6)mi_q7_16 = h1icsicogact6: forgets family member names (1/2);
# ** (7)mi_q8_16 = h1icsicogact7: forgets thoughts in middle of conversation (1/2);
# ** (8)mi_q9_16 = h1icsicogact8: difficulty finding the right words (1/2);
# ** (9)mi_q10_16 = h1icsicogact9:  uses wrong words (1/2);
# ** (10)mi_q11_16 = h1icsicogact10 : talks about past not present (1/2);
# ** (11)mi_q12_16 = h1icsicogact11: forgets when last saw informant (1/2);
# ** (12)mi_q13_16 = h1icsicogact12: forgets what happend yesterday;
# ** (13)mi_q14_16 = h1icsicogact13: forgets where s/he is ;
# ** (14)mi_q15_16 = h1icsicogact14: gets lost in community;
# ** (15)mi_q16_16 = h1icsicogact15: gets lost at home;
# ** (16)mi_q17_16 = h1i1066_1: difficulty with household chores;
# ** (17)mi_q18_16 = h1i1066_2: Loss of skill or hobby/Stopped doing activities/hobbies;
# ** (18)mi_q19_16 = h1i1066_3: difficulty handling money;
# ** (19)mi_q20_16 = h1i1066_4: difficulty adjusting to routine change;
# ** (20)mi_q22_16 = h1ibl2ea: ability to feed self;


mex_cog_df = mex_cog_df.rename(columns={
    'mi_q2_16': 'binf1csidmental',
    'mi_q3_16': 'binf1csidmemory',
    'mi_q4_16': 'binf1csidput',
    'mi_q5_16': 'binf1csidkept',
    'mi_q6_16': 'binf1csidfrdname',
    'mi_q7_16': 'binf1csidfamname',
    'mi_q8_16': 'binf1csidconvers',
    'mi_q9_16': 'binf1csidwordfind',
    'mi_q10_16': 'binf1csidwordwr',
    'mi_q11_16': 'binf1csidpast',
    'mi_q12_16': 'binf1csidlastsee',
    'mi_q13_16': 'binf1csidlastday',
    'mi_q14_16': 'binf1csidorient',
    'mi_q15_16': 'binf1csidlostout',
    'mi_q16_16': 'binf1csidlostin',
    'mi_q17_16': 'binf1chores',
    'mi_q18_16': 'binf1hobby',
    'mi_q19_16': 'binf1money',
    'mi_q20_16': 'binf1change',
    'mi_q22_16': 'binf1bl2feed'
})

csid_vars = [
    'binf1csidmental', 'binf1csidmemory', 'binf1csidput', 'binf1csidkept', 
    'binf1csidfrdname', 'binf1csidfamname', 'binf1csidconvers', 'binf1csidwordfind',
    'binf1csidwordwr', 'binf1csidpast', 'binf1csidlastsee', 'binf1csidlastday', 
    'binf1csidorient', 'binf1csidlostout', 'binf1csidlostin', 'binf1chores',
    'binf1hobby', 'binf1money', 'binf1change', 'binf1bl2feed'
]

mex_cog_df = mex_cog_df.applymap(lambda x: pd.NA if pd.isna(x) else x)

mex_cog_df[csid_vars] = mex_cog_df[csid_vars].replace({"8. Does not know  No sabe": pd.NA, "9. Refused  No responde": pd.NA})

mex_cog_df['mis_csid'] = mex_cog_df[csid_vars].isna().any(axis=1).astype(int)

# recode values to 0 (representing no decline/change) and 1 (representing decline/change)
vars_to_recode = [
    'binf1csidmental', 'binf1csidmemory', 'binf1csidput', 'binf1csidkept', 
    'binf1csidfrdname', 'binf1csidfamname', 'binf1csidconvers', 'binf1csidwordfind',
    'binf1csidwordwr', 'binf1csidpast', 'binf1csidlastsee', 'binf1csidlastday', 
    'binf1csidorient', 'binf1csidlostout', 'binf1csidlostin', 'binf1chores',
    'binf1hobby', 'binf1money', 'binf1change'
]

def recode_values(value):
    if pd.isna(value):
        return pd.NA
    if str(value).startswith('0.'):
        return 0
    if str(value).startswith('1.') or str(value).startswith('2.') or str(value).startswith('3.'):
        return 1
    return pd.NA

for var in vars_to_recode:
    mex_cog_df[var] = mex_cog_df[var].apply(recode_values)
    
mex_cog_df['binf1bl2feed'] = mex_cog_df['binf1bl2feed'].replace({"9. Does not know/Refused  No sabe/No responde":pd.NA}).map(
    {"0. Feeds himself cleanly with proper silverware  Come limpiamente con los cubiertos adecuados?":0, 
     "1. Feeds himself disorderly with a spoon  Come desordenadamente con una cuchara?":1,
     "2. Only eats simple solids like cookies  Solo alimentos solidos simples como una galleta?":1,
     "3. Has to be fed  Tiene que ser alimentado(a)?":1
     }
)



/var/folders/nh/hjw01w5j4k7csj4x6k02jv000000gn/T/ipykernel_20238/3205761526.py:55: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mex_cog_df = mex_cog_df.applymap(lambda x: pd.NA if pd.isna(x) else x)


In [37]:
# drop incomplete entries 

# Calculate row total for specific columns if mis_csid is 0
mex_cog_df['bcsid'] = np.where(
    mex_cog_df['mis_csid'] == 0,
    mex_cog_df[csid_vars].sum(axis=1),
    pd.NA
)

# Drop rows where mis_csid is 1
mex_cog_df = mex_cog_df[mex_cog_df['mis_csid'] != 1]

columns_to_check = ['binf1money', 'binf1change', 'binf1bl2feed']

mex_cog_df[columns_to_check] = mex_cog_df[columns_to_check].applymap(lambda x: pd.NA if pd.isna(x) else x)

# Drop rows with NA values in the specified columns
mex_cog_df = mex_cog_df.dropna(subset=columns_to_check)


/var/folders/nh/hjw01w5j4k7csj4x6k02jv000000gn/T/ipykernel_20238/1732339962.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mex_cog_df[columns_to_check] = mex_cog_df[columns_to_check].applymap(lambda x: pd.NA if pd.isna(x) else x)


In [38]:
# for var in csid_vars:
#     freq_table = mex_cog_df[mex_cog_df['mis_csid'] == 0][var].value_counts()
#     freq_table.to_csv(f'mhas_csid_freqtbl_{var}.txt', sep='\t', header=True)

# # Display frequency tables
# for var in csid_vars:
#     freq_table = mex_cog_df[mex_cog_df['mis_csid'] == 0][var].value_counts()
#     print(f"Frequency Table for {var}")
#     print(freq_table)
#     print("\n")

In [39]:
summary_vars = [
    'binf1csidmental', 'binf1csidmemory', 'binf1csidput', 'binf1csidkept', 
    'binf1csidfrdname', 'binf1csidfamname', 'binf1csidconvers', 'binf1csidwordfind',
    'binf1csidwordwr', 'binf1csidpast', 'binf1csidlastsee', 'binf1csidlastday', 
    'binf1csidorient', 'binf1csidlostout', 'binf1csidlostin', 'binf1chores',
    'binf1hobby', 'binf1money', 'binf1change', 'binf1bl2feed',
    'fgcp', 'fmem', 'fexf', 'flang', 'forient', 'mmse_25', 'bcsid',
    'rage', 'rfemale', 'reduc', 'iage', 'ifemale', 'ieduc', 
    'relation', 'ispouse', 'ichild', 'iothfam', 'inonfam', 'coresi'
]

for col in summary_vars:
    if mex_cog_df[col].dtype.name == 'category':
        mex_cog_df[col] = mex_cog_df[col].astype(float)

summary_stats = mex_cog_df[summary_vars].describe().transpose()
# Adding sum to the summary statistics
summary_stats['sum'] = mex_cog_df[summary_vars].sum()
summary_stats = summary_stats[['mean', 'sum', 'std', 'min', 'max']]

# Display summary statistics
print("Summary Statistics")
print(summary_stats)


Summary Statistics
                      mean       sum        std     min     max
binf1csidmental   0.322857     565.0   0.467702   0.000   1.000
binf1csidmemory   0.291429     510.0   0.454550   0.000   1.000
binf1csidput      0.586286    1026.0   0.492639   0.000   1.000
fgcp             -0.900582 -1576.019   0.969925  -3.585   1.801
fmem             -0.998330 -1747.078   0.823638  -3.318   1.134
mmse_25          17.682286   30944.0   4.650344   0.000  25.000
rfemale           0.580571    1016.0   0.493607   0.000   1.000
iage             50.792000     88886  17.060889  18.000  97.000
ifemale           0.672000    1176.0   0.469619   0.000   1.000
ispouse           0.425143       744   0.494506   0.000   1.000
ichild            0.428571       750   0.495013   0.000   1.000
iothfam           0.046857        82   0.211393   0.000   1.000
inonfam           0.099429       174   0.299322   0.000   1.000
coresi            0.798286    1397.0   0.401395   0.000   1.000


In [40]:
keep_vars = [
    'cunicah', 'np', 'id_mexcog', 'hrs', 'mex', 'wgt', 'binf1csidmental', 'binf1csidmemory', 'binf1csidput', 'binf1csidkept', 
    'binf1csidfrdname', 'binf1csidfamname', 'binf1csidconvers', 'binf1csidwordfind',
    'binf1csidwordwr', 'binf1csidpast', 'binf1csidlastsee', 'binf1csidlastday', 
    'binf1csidorient', 'binf1csidlostout', 'binf1csidlostin', 'binf1chores',
    'binf1hobby', 'binf1money', 'binf1change', 'binf1bl2feed',
    'fgcp', 'fmem', 'fexf', 'flang', 'forient', 'mmse_25', 'bcsid',
    'rage', 'rfemale', 'reduc', 'iage', 'ifemale', 'ieduc', 
    'relation', 'ispouse', 'ichild', 'iothfam', 'inonfam', 'coresi'
]

# Keep only the specified columns
mex_cog_df = mex_cog_df[keep_vars]

In [44]:
# mex_cog_df.to_csv('/Users/novak/ZhangYS Dropbox/Shared/HCAP/python file/mex-cog.csv', index=False)

In [42]:
mex_cog_df = mex_cog_df[mex_cog_df['rage'] >= 65]
mex_cog_df = mex_cog_df.dropna()

# List of variables
variables = ['rage', 'rfemale', 'reduc', 'iage', 'ifemale', 'ieduc', 
             'coresi', 'ispouse', 'ichild', 'iothfam', 'inonfam']

# Create an empty DataFrame to store the summary statistics
summary_table = pd.DataFrame()

# Loop through each variable and calculate the summary statistics
for var in variables:
    mex_cog_df[var] = pd.to_numeric(mex_cog_df[var], errors='coerce')
    summary_stats = mex_cog_df[var].describe()
    summary_table[var] = summary_stats

# Display the summary table
summary_table
